In [1]:
# Install libraries.

!pip install selenium
!pip install pymongo

In [ ]:
# Importing required libraries.

import time

from bs4 import BeautifulSoup
from selenium import webdriver

from pymongo import MongoClient

### TALLER 1 - CORTE 2 - PRIMERA PARTE 

### - Conexión al sitio web a analizar y manipulación del buscador para realizar scraping. 


In [5]:
# Base URL of the site to be analyzed.
SITE_URL = 'https://www.elespectador.com'

In [ ]:
# Chrome web driver path
# Download the driver for you S.O. here: https://chromedriver.storage.googleapis.com/index.html?path=94.0.4606.41/
DRIVER_PATH = './chromedriver.exe'

In [51]:
# Creating a new Chrome window.
browser = webdriver.Chrome(executable_path = DRIVER_PATH)

C:\Users\toshiba\AppData\Local\Temp/ipykernel_14224/911755350.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(executable_path = DRIVER_PATH)


In [52]:
def make_request(browser, relative_path):
    # Making the request and rendering the browser.
    browser.get(SITE_URL + relative_path)
    
    # Simulating vertical scrolling for handling lazy load.
    check_height = browser.execute_script('return document.body.scrollHeight;')
    while True:
        browser.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(3)
        height = browser.execute_script('return document.body.scrollHeight;')
        if height == check_height: 
            break 
        check_height = height
    
    # Getting HTML content and passing it to BeautifulSoup for scraping analysis.
    return BeautifulSoup(browser.page_source, 'html.parser')

### 1. Categoría de Salud.

In [4]:
# Creating a connection to MongoDB to create the health module. 
client = MongoClient('localhost', 27017)
db = client['news']
collection = db['elespectador-salud']

In [9]:
# Getting HTML content for news listing page.

news = []
url = '/archivo/salud'

for n in range(1, 12):
    soup = make_request(browser, url + str(n))
    # Finding the section where news are contained.
    layout = soup.find(class_ = 'Layout-flexAds')
    # Getting blocks from layout.
    blocks = layout.find('section').find_all(recursive = False)
    # Finding and concatenating news cards.
    cards = blocks[0].find_all(class_ = 'Card') + blocks[1].find_all(class_ = 'Card')
    # Building a list with title and relative path of the news founded.
    for card in cards:
        news.append({
            'title': card.find('h2', class_ = 'Card-Title').find('a').get_text(),
            'relative_path': card.find('h2', class_ = 'Card-Title').find('a')['href']
        })

    

In [10]:
for n in news:
    # Getting HTML content for news page.
    soup = make_request(browser, n['relative_path'])
    
    # Extracting news metadata.
    n['datetime'] = soup.find(class_ = 'ArticleHeader-Date').get_text()
    n['author'] = soup.find(class_ = 'ACredit-Author').get_text()
    n['summary'] = soup.find(class_ = 'ArticleHeader-Hook').find('div').get_text()
    
    # Extracting and concatenating news full text.
    paragraphs = soup.find_all(class_ = 'font--secondary')
    n['full_text'] = ' '.join([p.get_text() for p in paragraphs])

AttributeError: 'NoneType' object has no attribute 'get_text'

In [11]:
news

[{'title': 'No falte: este sábado 23 de octubre habrá jornada de vacunación en Colombia',
  'relative_path': '/salud/no-falte-este-sabado-23-de-octubre-habra-jornada-de-vacunacion-en-colombia/',
  'datetime': '21 oct. 2021  - 7:49 p.\xa0m.',
  'author': 'Redacción Salud',
  'summary': 'Durante la pandemia de COVID-19 miles de niños dejaron de recibir las vacunas que necesitan. En Colombia esté sábado será la oportunidad de ponerse al día.',
  'full_text': 'Aunque por estos días solo se hable de las vacunas para el COVID-19, hay otros esquemas que son indispensables, especialmente para los menores de edad: los que hacen parte del Plan Ampliado de Inmunización (PAI). De él dependen miles de vidas. (Lea todas las noticias de salud en Colombia) Por eso, este sábado se realizará una nueva jornada nacional de vacunación en la que el Estado colombiano espera vacunar a alrededor 396 mil niños menores de 6 años de edad, 55 mil gestantes y 10 mil adultos de 60 años y más. (Lea Entre 80.000 y 180

In [ ]:
# Storing extracted information for further analysis.
collection.insert_many(news)

### 2. Categoría de Ambiente.

In [85]:
# Creating a connection to MongoDB to create the environment module. 
client = MongoClient('localhost', 27017)
db = client['news']
collection = db['elespectador-ambiente']

In [14]:
# Getting HTML content for news listing page.

newsEnvironment = []
url = '/archivo/ambiente'

for n in range(1, 10):
    soup = make_request(browser, url + str(n))
    # Finding the section where news are contained.
    layout = soup.find(class_ = 'Layout-flexAds')
    # Getting blocks from layout.
    blocks = layout.find('section').find_all(recursive = False)
    # Finding and concatenating news cards.
    cards = blocks[0].find_all(class_ = 'Card') + blocks[1].find_all(class_ = 'Card')
    # Building a list with title and relative path of the news founded.
    for card in cards:
        newsEnvironment.append({
            'title': card.find('h2', class_ = 'Card-Title').find('a').get_text(),
            'relative_path': card.find('h2', class_ = 'Card-Title').find('a')['href']
        })


In [83]:
for n in newsEnvironment:
    # Getting HTML content for news page.
    soup = make_request(browser, n['relative_path'])
    
    # Extracting news metadata.
    n['datetime'] = soup.find(class_ = 'ArticleHeader-Date').get_text()
    n['author'] = soup.find(class_ = 'ACredit-Author').get_text()
    n['summary'] = soup.find(class_ = 'ArticleHeader-Hook').find('div').get_text()
    
    # Extracting and concatenating news full text.
    paragraphs = soup.find_all(class_ = 'font--secondary')
    n['full_text'] = ' '.join([p.get_text() for p in paragraphs])

WebDriverException: Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: chrome=94.0.4606.81)
Stacktrace:
Backtrace:
	Ordinal0 [0x00D1B9D3+2472403]
	Ordinal0 [0x00CB6961+2058593]
	Ordinal0 [0x00BC22B8+1057464]
	Ordinal0 [0x00BBEEB1+1044145]
	Ordinal0 [0x00BB4FE9+1003497]
	Ordinal0 [0x00BB5BF0+1006576]
	Ordinal0 [0x00BB523A+1004090]
	Ordinal0 [0x00BB496A+1001834]
	Ordinal0 [0x00BB3AFD+998141]
	Ordinal0 [0x00BB3DB6+998838]
	Ordinal0 [0x00BC388A+1063050]
	Ordinal0 [0x00C14F0D+1396493]
	Ordinal0 [0x00C05593+1332627]
	Ordinal0 [0x00C1495F+1395039]
	Ordinal0 [0x00C0545B+1332315]
	Ordinal0 [0x00BE1FB4+1187764]
	Ordinal0 [0x00BE2E09+1191433]
	GetHandleVerifier [0x00EA3396+1551078]
	GetHandleVerifier [0x00F4D53A+2247818]
	GetHandleVerifier [0x00DA7FBB+521995]
	GetHandleVerifier [0x00DA7079+518089]
	Ordinal0 [0x00CBBD4D+2080077]
	Ordinal0 [0x00CBFC18+2096152]
	Ordinal0 [0x00CBFD52+2096466]
	Ordinal0 [0x00CC94B1+2135217]
	BaseThreadInitThunk [0x7619FA29+25]
	RtlGetAppContainerNamed

In [84]:
newsEnvironment

[{'title': 'Vibrio, la bacteria indicadora del cambio climático que está en aumento',
  'relative_path': '/ambiente/que-es-la-vibrio-la-bacteria-indicadora-del-cambio-climatico-que-esta-en-aumento/',
  'datetime': '21 oct. 2021  - 12:10 p.\xa0m.',
  'author': 'María Mónica Monsalve',
  'summary': 'A este patógeno, que tiene una especie que causa el cólera, le atraen las aguas con baja salinidad y más calientes, ambos escenarios que están creciendo con el cambio climático. Luis Escobar y Jaime Martínez Urtanza, autores de la sección de enfermedades infecciosas de The Lancet Countdown, explican el panorama.',
  'full_text': '“La Vibrio es como un barómetro del cambio climático, un organismo modelo que sirve para establecer las señales de cambio”, es una de las primeras cosas que comenta Jaime Martínez Urtanza, parte del Departamento de Genética y Microbiología de la Universidad Autónoma de Barcelona, España, y uno de los más de 90 autores del “Reporte de Lancet Countdown 2021 sobre la sa

In [86]:
# Storing extracted information for further analysis.
collection.insert_many(newsEnvironment)

### 3. Categoría de Ciencia. 

In [87]:
# Creating a connection to MongoDB to create the science module. 
client = MongoClient('localhost', 27017)
db = client['news']
collection = db['elespectador-ciencia']

In [88]:
# Getting HTML content for news listing page.

newsScience = []
url = '/archivo/ciencia'

for n in range(1, 10):
    soup = make_request(browser, url + str(n))
    # Finding the section where news are contained.
    layout = soup.find(class_ = 'Layout-flexAds')
    # Getting blocks from layout.
    blocks = layout.find('section').find_all(recursive = False)
    # Finding and concatenating news cards.
    cards = blocks[0].find_all(class_ = 'Card') + blocks[1].find_all(class_ = 'Card')
    # Building a list with title and relative path of the news founded.
    for card in cards:
        newsScience.append({
            'title': card.find('h2', class_ = 'Card-Title').find('a').get_text(),
            'relative_path': card.find('h2', class_ = 'Card-Title').find('a')['href']
        })


In [114]:
for n in newsScience:
    # Getting HTML content for news page.
    soup = make_request(browser, n['relative_path'])
    
    # Extracting news metadata.
    n['datetime'] = soup.find(class_ = 'ArticleHeader-Date').get_text()
    n['author'] = soup.find(class_ = 'ACredit-Author').get_text()
    n['summary'] = soup.find(class_ = 'ArticleHeader-Hook').find('div').get_text()
    
    # Extracting and concatenating news full text.
    paragraphs = soup.find_all(class_ = 'font--secondary')
    n['full_text'] = ' '.join([p.get_text() for p in paragraphs])

In [115]:
newsScience

[{'title': 'Diana Valencia, la colombiana que estudia vida en los exoplanetas',
  'relative_path': '/ciencia/diana-valencia-la-colombiana-que-estudia-vida-en-los-exoplanetas/',
  'datetime': '21 oct. 2021  - 9:00 p.\xa0m.',
  'author': 'Paula Casas Mogollón',
  'summary': 'La astrofísica planetaria fue reconocida con el Premio Paolo Farinella 2021, otorgado por la Sociedad Planetaria Europea, por ser una de las científicas que más conocen sobre exoplanetas, aquellos que no son parte del sistema solar.',
  'full_text': 'En la vida de Diana Valencia las matemáticas siempre han estado presentes. Desde los 11 años, cuando estaba en el colegio, asistía a olimpiadas matemáticas, pero cuando llegó el momento de escoger una carrera prefirió la física. “Pensé en enfocarme en la rama de matemáticas, pero, si te digo la verdad, dije: de pronto está muy difícil”, cuenta entre risas. “A mí siempre me causó curiosidad entender el mundo físico, cómo funciona y eso me lo ha permitido mi profesión”. (L

In [117]:
# Storing extracted information for further analysis.
collection.insert_many(newsScience)

### 4. Categoría de Tecnología.

In [118]:
# Creating a connection to MongoDB to create the technology module. 
client = MongoClient('localhost', 27017)
db = client['news']
collection = db['elespectador-tecnologia']

In [119]:
# Getting HTML content for news listing page.

newsTechnology = []
url = '/archivo/tecnologia'

for n in range(1, 10):
    soup = make_request(browser, url + str(n))
    # Finding the section where news are contained.
    layout = soup.find(class_ = 'Layout-flexAds')
    # Getting blocks from layout.
    blocks = layout.find('section').find_all(recursive = False)
    # Finding and concatenating news cards.
    cards = blocks[0].find_all(class_ = 'Card') + blocks[1].find_all(class_ = 'Card')
    # Building a list with title and relative path of the news founded.
    for card in cards:
        newsTechnology.append({
            'title': card.find('h2', class_ = 'Card-Title').find('a').get_text(),
            'relative_path': card.find('h2', class_ = 'Card-Title').find('a')['href']
        })

In [120]:
for n in newsTechnology:
    # Getting HTML content for news page.
    soup = make_request(browser, n['relative_path'])
    
    # Extracting news metadata.
    n['datetime'] = soup.find(class_ = 'ArticleHeader-Date').get_text()
    n['author'] = soup.find(class_ = 'ACredit-Author').get_text()
    n['summary'] = soup.find(class_ = 'ArticleHeader-Hook').find('div').get_text()
    
    # Extracting and concatenating news full text.
    paragraphs = soup.find_all(class_ = 'font--secondary')
    n['full_text'] = ' '.join([p.get_text() for p in paragraphs])

In [121]:
newsTechnology

[{'title': 'Google bajará comisión a suscripciones en tienda de aplicaciones',
  'relative_path': '/tecnologia/google-bajara-comision-a-suscripciones-en-tienda-de-aplicaciones/',
  'datetime': '21 oct. 2021  - 4:27 p.\xa0m.',
  'author': 'Agencia Bloomberg',
  'summary': 'A partir del 1 de enero, Google Play Store cobrará a las aplicaciones de suscripción de terceros una comisión del 15 %, informó la compañía el jueves. Anteriormente, a las aplicaciones de suscripción se les cobraba un 30 %.',
  'full_text': 'Google reducirá las comisiones que cobra por los servicios de suscripción en su tienda de aplicaciones tras la presión de los desarrolladores y los legisladores. A partir del 1 de enero, Google Play Store cobrará a las aplicaciones de suscripción de terceros una comisión del 15%, informó la compañía el jueves. Anteriormente, a las aplicaciones de suscripción se les cobraba un 30% durante el primer año y luego un 15%. Google también cobraba el 15% por el primer US$1 millón de ingre

In [122]:
# Storing extracted information for further analysis.
collection.insert_many(newsTechnology)

### 5. Categoría de Economía. 


In [123]:
# Creating a connection to MongoDB to create the economy module. 
client = MongoClient('localhost', 27017)
db = client['news']
collection = db['elespectador-economia']

In [125]:
# Getting HTML content for news listing page.

newsEconomy = []
url = '/archivo/economia'

for n in range(1, 10):
    soup = make_request(browser, url + str(n))
    # Finding the section where news are contained.
    layout = soup.find(class_ = 'Layout-flexAds')
    # Getting blocks from layout.
    blocks = layout.find('section').find_all(recursive = False)
    # Finding and concatenating news cards.
    cards = blocks[0].find_all(class_ = 'Card') + blocks[1].find_all(class_ = 'Card')
    # Building a list with title and relative path of the news founded.
    for card in cards:
        newsEconomy.append({
            'title': card.find('h2', class_ = 'Card-Title').find('a').get_text(),
            'relative_path': card.find('h2', class_ = 'Card-Title').find('a')['href']
        })

In [126]:
for n in newsEconomy:
    # Getting HTML content for news page.
    soup = make_request(browser, n['relative_path'])
    
    # Extracting news metadata.
    n['datetime'] = soup.find(class_ = 'ArticleHeader-Date').get_text()
    n['author'] = soup.find(class_ = 'ACredit-Author').get_text()
    n['summary'] = soup.find(class_ = 'ArticleHeader-Hook').find('div').get_text()
    
    # Extracting and concatenating news full text.
    paragraphs = soup.find_all(class_ = 'font--secondary')
    n['full_text'] = ' '.join([p.get_text() for p in paragraphs])

In [127]:
newsEconomy

[{'title': 'Caos en la cadena de suministro amenaza el crecimiento económico mundial',
  'relative_path': '/economia/caos-en-la-cadena-de-suministro-amenaza-el-crecimiento-economico-mundial/',
  'datetime': '21 oct. 2021  - 9:50 p.\xa0m.',
  'author': 'Agencia Anadolu',
  'summary': 'El aumento de los precios de las materias primas, la congestión de los puertos y la escasez de conductores de camiones, entre otros, afectan a las economías de todos los países y los problemas no serán superados sino hasta 2023, cuando menos.',
  'full_text': 'Mientras la economía mundial se recupera de la crisis por la pandemia de coronavirus, las interrupciones en la cadena de suministro en curso ponen en riesgo el crecimiento mundial. Los problemas en la cadena de suministro, incluidos el aumento de los precios de las materias primas, la escasez de chips de computador, la congestión de los puertos y la escasez de conductores de camiones, amenazan la economía mundial.  En declaraciones a la Agencia Anado

In [128]:
# Storing extracted information for further analysis.
collection.insert_many(newsEconomy)